# Running TIMES Models using Python Notebooks

This is a demonstration of how to use Python (Jupyter) notebooks to define, run, modify, and analyze TIMES model scenarios.

Using `xl2times`'s new `TimesModel` API, one can run model scenarios in an interactive, reproducible, self-documenting, and programmable way.

## Setup the Demo on Colab

This section is not neccessary if you have installed `xl2times` and the Demo models on your local machine.

In [1]:
%pip install gdx2py gamspy-base git+https://github.com/etsap-TIMES/xl2times.git

  Cloning https://github.com/etsap-TIMES/xl2times.git to /tmp/pip-req-build-8gk94_kv
  Running command git clone --filter=blob:none --quiet https://github.com/etsap-TIMES/xl2times.git /tmp/pip-req-build-8gk94_kv
  Resolved https://github.com/etsap-TIMES/xl2times.git to commit c6e062d74bea0928aa6b0c23ff3b292dbecad0f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.7 MB/s eta 0:00:00
  Created wheel for xl2times: filename=xl2times-0.2.2-py3-none-any.whl size=80618 sha256=6b89a28345426554dc4b506486e835122fbf6f37a5f813af1a24800bca68b2f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-qpp9r57n/wheels/d4/b5/63/8fcb20cf732167e710e5117b2dc5eb8154d9972bbf164fb9c1
Successfully built xl2times


In [2]:
%%bash
# Download the Demo 3 model
curl 'https://drive.usercontent.google.com/download?id=10zeI0xDk4g_I1e5xFwiC7e74Z21uYgWN&export=download&authuser=0&confirm=t' -o DemoS_003.zip
unzip DemoS_003.zip
# Clone the TIMES source code
git clone --filter=blob:none https://github.com/etsap-TIMES/TIMES_model TIMES_model
pushd TIMES_model/
git checkout b488fb07f0899ee8b7e710c230b1a9414fa06f7d
popd

Archive:  DemoS_003.zip
   creating: DemoS_003/
  inflating: DemoS_003/BY_Trans.xlsx  
  inflating: DemoS_003/Sets-DemoModels.xlsx  
  inflating: DemoS_003/VT_REG_PRI_V03.xlsx  
   creating: DemoS_003/AppData/
  inflating: DemoS_003/SysSettings.xlsx  
  inflating: DemoS_003/AppData/ResultViews.json  
  inflating: DemoS_003/AppData/Cases.json  
  inflating: DemoS_003/AppData/FrontEndFormFormSettings.json  
  inflating: DemoS_003/AppData/Groups.json  
   creating: DemoS_003/AppData/SolverOptFiles/
  inflating: DemoS_003/AppData/SolverOptFiles/CPLEX.OPT  
/content/TIMES_model /content
/content


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  226k  100  226k    0     0  61418      0  0:00:03  0:00:03 --:--:-- 61414
Cloning into 'TIMES_model'...
Note: switching to 'b488fb07f0899ee8b7e710c230b1a9414fa06f7d'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at b488fb0 Create CODEOWNERS


## Run a model scenario

In [3]:
from os import path

import gdx2py
from gamspy_base import directory

import xl2times.main as xt
from xl2times.utils import run_gams, setup_logger

In [4]:
setup_logger(level=0)
input_dir = "/content/DemoS_003/"
output_dir = "/content/DemoS_003-all-out/"
times_dir = "/content/TIMES_model/"

model, config = xt.read_xl([input_dir], regions="", include_dummy_imports=True)

/usr/local/lib/python3.11/dist-packages/xl2times/transforms.py:1276: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'EOH' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[
Applying transformations from ~TFM_UPD in SYSSETTINGS: 100%|██████████| 2/2 [00:00<00:00, 67.39it/s]


In [5]:
# Inspect some key parameters in the solution
def inspect_solution():
    with gdx2py.GdxFile(path.join(output_dir, "scenario.gdx"), gams_dir=directory) as g:
        print(g["REG_OBJ"].values())
        print(g["PAR_COMPRDL"].values())
        print(g["EQE_COMPRD"])

In [6]:
print(model.attributes.query('attribute == "COM_IE"')["value"])
tables = xt.to_tables(config, model)
xt.write_dd_files(tables, config, output_dir)

run_gams(times_dir, output_dir)

inspect_solution()

169    0.9
Name: value, dtype: object
2025-06-07 11:17:29.773 |  SUCCESS : Excel files successfully converted to DD and written to /content/DemoS_003-all-out/
2025-06-07 11:17:30.729 |  SUCCESS : Ran GAMS successfully on /content/DemoS_003-all-out/:
*** Status: Normal completion--- Job scenario.run Stop 06/07/25 11:17:30 elapsed 0:00:00.922
dict_values([3195557.668833001])
dict_values([])
None


## Modify some assumptions and re-run the scenario

One can think of extending this to programmatically run multiple scenarios, automatically varying a parameter from a set of options, etc.

In [7]:
# Modify COM_IE and re-run the scenario
com_ie = model.attributes.query('attribute == "COM_IE"')
model.attributes.loc[com_ie.index, "value"] = 0.95
print(model.attributes.query('attribute == "COM_IE"')["value"])

tables = xt.to_tables(config, model)
xt.write_dd_files(tables, config, output_dir)

run_gams(times_dir, output_dir)

inspect_solution()

169    0.95
Name: value, dtype: object
2025-06-07 11:17:31.791 |  SUCCESS : Excel files successfully converted to DD and written to /content/DemoS_003-all-out/
2025-06-07 11:17:32.413 |  SUCCESS : Ran GAMS successfully on /content/DemoS_003-all-out/:
*** Status: Normal completion--- Job scenario.run Stop 06/07/25 11:17:32 elapsed 0:00:00.595
dict_values([3095381.6789671676])
dict_values([])
None
